In [2]:
import serial
import plotly
#import matplotlib as plt
import sys
import visa
from LQCE.zvl13 import *
from LQCE.ncs513 import *
from LQCE.N5173B import *
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.cm as cm
from matplotlib.backends.backend_pdf import PdfPages
import time
import scipy.signal
%matplotlib 


Using matplotlib backend: Qt5Agg


In [26]:
gen = N5173B_gen("TCPIP0::192.168.1.50::INSTR")
#gen.set_modulation_state(0)
gen.set_frequency(2.5,"GHz")
gen.set_power(-10)
float(gen.get_power())
gen.set_output_state("OFF")
gen.get_output_state()
for i in range(10):
    gen.set_frequency(0.1*(i+1), "GHz")
    time.sleep(1)
    gen.set_power(-20 + i)
    

In [90]:
data_averaged = np.average(data,0)
peak_data = scipy.signal.find_peaks(data_averaged, height=4, distance=100)
print(peak_data[0])
print(x_axis[peak_data[0][0]])#,x_axis[peak_data[0][1]],x_axis[peak_data[0][2]], x_axis[peak_data[0][3]],x_axis[peak_data[0][4]],x_axis[peak_data[0][5]])


[50]
2500000000.0


In [5]:
nwa = zvl13("TCPIP0::192.168.1.46::INSTR")
ncs = Nsk_current_source("COM32")

In [16]:
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-50,"nop":101,"averages": 1, "cf_span":[2.5e9, 2e9], "bandwidth":10, "meas_counts":1})

ncs.enable_channel(1)
ncs.set_output_state(1,1)
ncs.set_range(1,"10mA")
ncs.set_current(1,0)
nwa.set_timeout(200000)


plt.ion()

fig = plt.figure()
z_upd = []

for i in range(10):
    ncs.set_current(1,i*1e-4)
    nwa.trig_meas()
    x_axis = nwa.get_X_axis_values()   
    data = nwa.get_data(int(nwa.get_meas_counts()))
    x_upd = x_axis#[]
    y_upd = []
    nx = 101#x_upd.shape[0]
    no_labels = 5
    step_x = int(nx/(no_labels -1))
    x_positions = np.arange(0,nx,step_x)
    x_labels = x_upd[::step_x]
    plt.yticks(x_positions, x_labels)
    
    z_upd.append(data[0][0])
    y_upd.append(i)
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
    #nwa.set_timeout(1000)
    #plt.plot(x_axis, np.average(data[0],0))
    #line1.set_ydata(data[0][i][50])
    fig.canvas.draw()
    fig.canvas.flush_events()


In [57]:
#ncs = Nsk_current_source("COM32")
#ncs.set_output_state(1,1)
ncs.close()



In [10]:
act_data = data[0]
len(x_axis)

101

In [11]:
from __future__ import division
from matplotlib.patches import Patch
from pylab import *


y = np.linspace(0,1e-3,5)
x = x_axis
X,Y = meshgrid(y, x)


Z = np.transpose(np.array(act_data))

pcolor(X, Y, Z, cmap=cm.RdBu, vmax=abs(Z).max(), vmin=-abs(Z).max())
colorbar()
#axis([-3,3,-3,3])

plt.ion()

    fig.canvas.draw()
    fig.canvas.flush_events()



In [74]:

import matplotlib.animation as animation


fig2 = plt.figure()
x = x_axis
y = [np.arange(3)]
base = np.array(act_data[0:3])
X,Y = meshgrid(x, y)
ims = []
plt.pcolormesh(X, Y, base)




In [4]:
x = np.arange(100)
y = np.arange(100)
temp = [[0]*len(x)]*len(y)
nrows, ncols = 100, 100
grid = temp #.reshape((nrows, ncols))

for i in range(len(x)):
    temp[i] = np.random.rand(100)
fig = plt.figure()
plt.imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', aspect='equal', cmap=cm.jet)
plt.colorbar()

plt.ion()
x_upd = []
y_upd = np.array([])
z_upd = []

for i in range(10):    
    time.sleep(1)
    #z_upd.append([1,2,3,4])
    z_upd.append(np.random.rand(4))
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
    
    fig.canvas.draw()
    fig.canvas.flush_events()    



In [2]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result
